In [4]:
import os
# import pickle
import argparse
import torch
import numpy as np
from math import ceil
from model_vc import Generator


device = 'cuda:0'

def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

def inference(config):
    G = Generator(config.dim_neck, config.dim_emb, config.dim_pre, config.freq).eval().to(device)
    cp_path = os.path.join(config.save_dir, "weights")
    if os.path.exists(cp_path):
        save_info = torch.load(cp_path)
        G.load_state_dict(save_info["model"])

    emb_org = np.load(config.emb_org)
    emb_org = np.reshape(emb_org, (-1))
    emb_trg = np.load(config.emb_trg)
    emb_trg = np.reshape(emb_trg, (-1))

    x_org = np.load(config.spectrogram_path).T
    x_org, len_pad = pad_seq(x_org)

    with torch.no_grad():
        _, x_identic_psnt, _ = G(x_org, emb_org, emb_trg)
        if len_pad == 0:
            x_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else:
            x_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()

    np.save("result.npy", x_trg.T)
    print("result saved.")
        

# g_checkpoint = torch.load('autovc.ckpt')
# G.load_state_dict(g_checkpoint['model'])
# def pad_seq(x, base=32):
#     len_out = int(base * ceil(float(x.shape[0])/base))
#     len_pad = len_out - x.shape[0]
#     assert len_pad >= 0
#     return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

# device = 'cuda:0'
# G = Generator(32,256,512,32).eval().to(device)

# g_checkpoint = torch.load('autovc.ckpt')
# G.load_state_dict(g_checkpoint['model'])

# metadata = pickle.load(open('metadata.pkl', "rb"))

# spect_vc = []

# for sbmt_i in metadata:
             
#     x_org = sbmt_i[2]
#     x_org, len_pad = pad_seq(x_org)
#     uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device)
#     emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device)
    
#     for sbmt_j in metadata:
                   
#         emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        
#         with torch.no_grad():
#             _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
#         if len_pad == 0:
#             uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
#         else:
#             uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        
#         spect_vc.append( ('{}x{}'.format(sbmt_i[0], sbmt_j[0]), uttr_trg) )
# with open('results.pkl', 'wb') as handle:
#     pickle.dump(spect_vc, handle)


if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Model configuration.
    parser.add_argument('--lambda_cd', type=float, default=1, help='weight for hidden code loss')
    parser.add_argument('--dim_neck', type=int, default=16)
    parser.add_argument('--dim_emb', type=int, default=2000)
    parser.add_argument('--dim_pre', type=int, default=512)
    parser.add_argument('--freq', type=int, default=16)
    
    # Training configuration.
    parser.add_argument('--spectrogram_path', type=str, default='/root/timbre/piano_test.npy')
    parser.add_argument('--emb_org', type=str, default='/root/timbre/data_syn/entries/piano_entry_embeddings.npy')
    parser.add_argument('--emb_trg', type=str, default='/root/timbre/data_syn/entries/guitar_entry_embeddings.npy')
    # parser.add_argument('--num_iters', type=int, default=1000000, help='number of total iterations')
    # parser.add_argument('--len_crop', type=int, default=128, help='dataloader output sequence length')
    
    # Miscellaneous.
    parser.add_argument('--save_dir', type=str, default="/root/timbre/autovc_cp")

    config = parser.parse_args()
    print(config)
    inference(config)

usage: ipykernel_launcher.py [-h] [--lambda_cd LAMBDA_CD]
                             [--dim_neck DIM_NECK] [--dim_emb DIM_EMB]
                             [--dim_pre DIM_PRE] [--freq FREQ]
                             [--spectrogram_path SPECTROGRAM_PATH]
                             [--emb_org EMB_ORG] [--emb_trg EMB_TRG]
                             [--save_dir SAVE_DIR]
ipykernel_launcher.py: error: argument --freq: invalid int value: '/tmp/tmp-15064tbJG5doaoAH0.json'


AssertionError: 